In [1]:
%matplotlib inline


Autograd(自动微分): Automatic Differentiation
===================================

PyTorch中所有神經網絡的核心即是 ``autograd`` 軟件包。
让我们先简要地介绍一下，然后再来训练第一个神经网络。


``autograd`` 软件包为Tensor上的所有操作提供自动微分。 
这是一个按运行定义(define-by-run)的框架，这意味着您的反向传播(backprop)是由代码的运行方式定义的，并且每次迭代都可以不同。

让我们通过一些例子来更简单地了解这一点。

Tensor(张量)
--------

``torch.Tensor`` 是軟件包的中心类別。 
如果将其属性``.requires_grad``设置为``True``，它将开始追踪对其的所有操作。
完成计算后，可以调用 ``.backward()``并自动计算所有梯度。 该张量的梯度将累积到``.grad``属性中。

若要使张量停止追踪历史记录，您可以调用``.detach()``将其从计算历史记录中分离出来，并防止将来的计算被追踪。

若要防止追踪历史记录（和占用内存），您还可以将代码块包装在``with torch.no_grad():``中。    
这在评估模型时特别有用，因为该模型可能具有带有`requires_grad=True`的可训练参数，但是我们不需要梯度。


还有另外一个类别对实现autograd非常重要-称为``Function``。

``Tensor``和``Function``相互连接并建立一个非循环图形(acyclic graph)，它编码完整的计算历史记录。     
每个张量都有一个``.grad_fn``属性，该属性引用创建了``Tensor``的``Function``（用户创建的Tensors除外-它们的``grad_fn``為``None``）。

如果要计算导数，可以在``Tensor``上调用``.backward()``。     
如果``Tensor``是标量（即，它保存一个元素数据），则无需为``.backward()``指定任何参数，但是，如果它具有更多元素，则需要指定``gradient``实际参数(argument)是匹配形状的张量。

譯註：  
1. `requires_grad`默認為False，即「不需要求導」    
2. 在张量间的计算过程中，如果在所有输入中，有任一输入需要求导，那么输出則会需要求导   
3. 在PyTorch中，``Tensor``与``tensor``不一样，须注意


In [2]:
import torch

创建一个张量并设置``require_grad = True``来追踪它的计算



In [3]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


进行一次张量运算：



In [4]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


``y``是由于操作而创建的，因此具有``grad_fn``。


In [5]:
print(y.grad_fn)

对``y``进行更多操作



In [6]:
z = y * y * 3
out = z.mean()

print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


``.requires_grad_( ... )``就地(in-place)更改现有Tensor的``requires_grad``标志。 如果未给出输入标志，则默认为False。



In [7]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True


Gradients(梯度)
---------
让我们现在进行反向传播。    
因为``out``包含单个标量，所以``out.backward()``等同于``out.backward(torch.tensor(1))``。

In [8]:
out.backward()

打印梯度) d(out)/dx




In [9]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


您应该会得到一个``4.5``的矩阵。     
让我们称呼``out``*张量*为“$o$”.     
$o = \frac{1}{4}\sum_i z_i$,
$z_i = 3(x_i+2)^2$ 且 $z_i\bigr\rvert_{x_i=1} = 27$.
因此,
$\frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i+2)$, 所以
$\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$.



您可以使用autograd做许多疯狂的事情！



In [10]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

tensor([ 833.0659, -209.5000, -988.5157], grad_fn=<MulBackward0>)


In [11]:
gradients = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(gradients)

print(x.grad)

tensor([2.0480e+02, 2.0480e+03, 2.0480e-01])


You can also stop autograd from tracking history on Tensors with ``.requires_grad=True`` by wrapping the code block in ``with torch.no_grad()``:
您还可以通过将代码块包装在``with torch.no_grad()``中来阻止autograd追踪带有``.requires_grad=True``的张量的历史纪录。



In [12]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
	print((x ** 2).requires_grad)

True
True
False


**稍后阅读:**

``autograd``和``Function``的文档位于
https://pytorch.org/docs/autograd

